In [45]:
# This script demonstrates the capabilities of cluster_slug by using
# it to re-analyze the M31 star clusters catalog of Johnson et al. (2015, ApJ)

# Import the libraries we'll need
from astropy.io import ascii
from astropy.io import fits
import emcee
from astropy import table
from astropy.table import unique, Table, hstack, Column
import sys
from numpy import inf
import numpy as np
import numpy.ma as ma
import time
from matplotlib.pyplot import figure as fig
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import curve_fit
from scipy.stats import norm
from slugpy.cluster_slug import cluster_slug
from slugpy import * 
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.ticker import AutoMinorLocator

In [61]:
apdata = Table.read('apdata-cluster_6phot+agemass.fits')

m31_Prop6filt=Table.read('/Volumes/My_Passport_for_Mac/M33_M31_LGCS_Files/M31-Andromeda/Prop_Tables/M31_Prop6filt.fits')
m31_Prop5filt=Table.read('/Volumes/My_Passport_for_Mac/M33_M31_LGCS_Files/M31-Andromeda/Prop_Tables/M31_Prop5filt891cls.fits')
m31_Prop4filt=Table.read('/Volumes/My_Passport_for_Mac/M33_M31_LGCS_Files/M31-Andromeda/Prop_Tables/M31_Prop4filt1197cls.fits')
m31_Prop2filt=Table.read('/Volumes/My_Passport_for_Mac/M33_M31_LGCS_Files/M31-Andromeda/Prop_Tables/M31_Prop2filt1867cls.fits')
m31_Prop6filt

ID,BEST_METHOD,Chi_2(6),Mass_PDF_P25(6),Mass_PDF_Median(6),Mass_PDF_P75(6),MassIQR(6),SlugMass-m31BestMass(6),Age_PDF_P25(6),Age_PDF_Median(6),Age_PDF_P75(6),AgeIQR(6),SlugAge-m31BestAge(6),AV_PDF_P25(6),A_V_PDF_Median(6),A_V_PDF_P75(6),AvIQR(6),SlugAV-CMD_AV_Best(6)
int32,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,I,2.7985711649757494,2.307080791931832,4.220375004918627,4.480205083225476,2.173124291293644,-0.08965105167155674,7.07086623612586,8.67716512372204,8.960629633297836,1.8897633971719765,0.29379263232044117,0.26388315860375433,0.6098587952263942,1.6477857050943139,1.3839025464905594,-0.6098587952263942
4,I,2.1949285655453195,2.2834598757221185,4.196754088708913,4.432963250806049,2.1495033750839303,-0.07106935054973373,7.07086623612586,8.645669067102506,8.960629633297836,1.8897633971719765,0.3290108797969076,0.21445806765766293,0.6098587952263942,1.6477857050943139,1.433327637436651,-0.6098587952263942
5,M,1.9879533094109163,2.543289954028967,3.440884769998081,3.7479566807243567,1.2046667266953897,-0.029632509789951023,7.637795255277452,8.267716387668111,8.67716512372204,1.0393698684445871,0.13228323086216243,0.31330824954984576,0.708708977118577,1.3512351594177652,1.0379269098679194,-0.258708989039506
6,M,0.18563253805191895,3.039329194432951,3.6770939320952163,4.007786759031205,0.9684575645982543,0.14977197938854836,7.385826802321189,8.173228217809513,8.551180897243908,1.1653540949227192,0.3267717821904874,0.26388315860375433,0.708708977118577,1.6477857050943139,1.3839025464905594,-0.20870897711857705
16,M,0.7210336510758213,2.4251853729804,3.322780188949513,3.9605449266117785,1.5353595536313787,0.24778360423285983,7.228346519223525,7.574803142038387,8.425196670765775,1.1968501515422503,0.8251964764918869,0.41215843144202857,1.0052595227951253,1.9443362507708621,1.5321778193288336,-0.2552595227951253
24,M,4.687324060279692,2.401564456770686,3.039329194432951,3.4172638537883673,1.015699397017681,0.07980809270694156,6.188976650778938,7.543307085418854,7.795275538375117,1.6062988875961794,-0.34330727615371703,0.31330824954984576,0.708708977118577,2.2903118873935018,1.977003637843656,-0.00870898903950601
25,M,0.6250095985542866,2.3779435405609726,3.440884769998081,3.77157759693407,1.3936340563730973,-0.0006179223876072726,7.669291311896986,8.425196670765775,8.897637520058769,1.228346208161783,0.07480332923422495,0.31330824954984576,0.7581340680646683,1.4500853413099482,1.1367770917601023,-0.2581340680646683
26,I,0.9506049565253976,2.4960481216095403,4.031407675240919,4.432963250806049,1.9369151291965085,-0.08405693480817966,7.511811028799321,8.897637520058769,9.464566539210363,1.9527555104110421,-0.34402957938494083,0.31330824954984576,0.8075591590107598,1.796060977932588,1.4827527283827422,-0.8075591590107598
27,M,0.45238064111523446,3.0865710268523783,3.488126602417508,3.9133030941923512,0.826732067339973,-0.16846951509145836,7.07086623612586,7.29133863246259,7.606299198657919,0.5354329625320595,0.2086613675374096,0.26388315860375433,0.6098587952263942,1.1535347956333997,0.8896516370296453,-0.10985879522639419


In [62]:
m31_Prop5filt

ID,BEST_METHOD,Chi_2(5),Mass_PDF_P25(5),Mass_PDF_Median(5),Mass_PDF_P75(5),MassIQR(5),SlugMass-m31BestMass(5),Age_PDF_P25(5),Age_PDF_Median(5),Age_PDF_P75(5),AgeIQR(5),SlugAge-m31BestAge(5),AV_PDF_P25(5),A_V_PDF_Median(5),A_V_PDF_P75(5),AvIQR(5),SlugAV-CMD_AV_Best(5)
int32,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,I,0.3194347026193026,2.354322624351259,4.0550285914506325,4.432963250806049,2.07864062645479,0.07569536179643777,7.07086623612586,8.393700614146244,8.929133576678304,1.8582673405524437,0.5772571418962364,0.26388315860375433,0.6592838861724857,1.7466358869864964,1.4827527283827422,-0.6592838861724857
4,I,0.22185885679230938,2.3779435405609726,3.9605449266117785,4.362100502176908,1.9841569616159354,0.1651398115474012,7.039370179506327,8.299212444287644,8.866141463439238,1.8267712839329109,0.6754675026117702,0.26388315860375433,0.6592838861724857,1.7466358869864964,1.4827527283827422,-0.6592838861724857
5,M,1.434794904162243,2.543289954028967,3.322780188949513,3.7479566807243567,1.2046667266953897,0.08847207125861667,7.669291311896986,8.14173216118998,8.708661180341572,1.0393698684445862,0.25826745734029366,0.26388315860375433,0.6592838861724857,1.2523849775255826,0.9885018189218283,-0.20928389809341463
6,M,0.12314988569737861,2.89760369717467,3.70071484830493,4.243995921128341,1.3463922239536705,0.12615106317883473,7.354330745701656,8.204724274429045,8.834645406819703,1.4803146611180473,0.2952757255709546,0.26388315860375433,0.6592838861724857,1.4995104322560393,1.2356272736522849,-0.15928388617248568
11,I,0.267115667537693,2.4251853729804,3.440884769998081,3.889682177982638,1.4644968050022382,-0.027481258950717624,7.669291311896986,8.36220455752671,9.023621746536902,1.354330434639916,0.47673606289565384,0.31330824954984576,0.7581340680646683,1.4006602503638568,1.087352000814011,-0.7581340680646683
16,M,0.17398336182454868,2.732257283706675,4.291237753547768,4.763656077742038,2.0313987940353626,-0.7206739603653949,7.322834689082123,8.929133576678304,9.590550765688494,2.267716076606371,-0.5291339581480301,0.26388315860375433,0.708708977118577,1.5983606141482223,1.334477455544468,0.041291022881422945
24,M,5.232476903302763,2.354322624351259,3.0629501106426646,3.4645056862077945,1.1101830618565356,0.05618717649722793,6.188976650778938,7.543307085418854,7.858267651614183,1.669291000835245,-0.34330727615371703,0.31330824954984576,0.8569842499568513,2.4880122511778677,2.174704001628022,-0.15698426187778025
25,M,0.3594941844475863,2.401564456770686,3.2755383565300864,3.7479566807243567,1.3463922239536705,0.16472849108038723,7.637795255277452,8.236220331048578,8.897637520058769,1.2598422647813168,0.2637796689514218,0.26388315860375433,0.708708977118577,1.4006602503638568,1.1367770917601026,-0.20870897711857705
26,I,0.7651269674377409,3.8660612617729244,4.456584167015762,4.787276993951751,0.921215732178827,-0.5092334265830232,8.67716512372204,9.496062595829896,9.905511331883824,1.2283462081617849,-0.9424546551560677,0.26388315860375433,0.5604337042803027,1.301810068471674,1.0379269098679198,-0.5604337042803027


In [63]:
m31_Prop4filt

ID,BEST_METHOD,Chi_2(4),Mass_PDF_P25(4),Mass_PDF_Median(4),Mass_PDF_P75(4),MassIQR(4),SlugMass-m31BestMass(4),Age_PDF_P25(4),Age_PDF_Median(4),Age_PDF_P75(4),AgeIQR(4),SlugAge-m31BestAge(4),AV_PDF_P25(4),A_V_PDF_Median(4),A_V_PDF_P75(4),AvIQR(4),SlugAV-CMD_AV_Best(4)
int32,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,I,2.5684339984198012,2.401564456770686,4.267616837338054,4.527446915644903,2.125882458874217,-0.136892884090984,7.1338583493649255,8.708661180341572,8.99212568991737,1.8582673405524437,0.26229657570090836,0.26388315860375433,0.6098587952263942,1.548935523202131,1.2850523645983767,-0.6098587952263942
2,M,0.28868409604292605,2.259838959512405,3.440884769998081,3.818819429353497,1.5589804698410923,0.5379913438935207,7.543307085418854,8.267716387668111,8.740157236961105,1.1968501515422512,0.13228323086216243,0.26388315860375433,0.6592838861724857,1.4500853413099482,1.186202182706194,0.4907160899856564
3,I,0.22965961055129638,2.3779435405609726,3.0865710268523783,3.4172638537883673,1.0393203132273947,0.0036339272949605927,7.669291311896986,8.236220331048578,8.614173010482974,0.9448816985859878,0.332355528021246,0.21445806765766293,0.6098587952263942,1.3512351594177652,1.1367770917601023,-0.6098587952263942
4,I,1.8311898052344433,2.354322624351259,4.243995921128341,4.50382599943519,2.1495033750839307,-0.11831118296916099,7.1338583493649255,8.67716512372204,8.99212568991737,1.8582673405524437,0.29751482317737477,0.21445806765766293,0.5604337042803027,1.548935523202131,1.334477455544468,-0.5604337042803027
5,M,0.3931113872110326,2.259838959512405,3.322780188949513,3.6770939320952163,1.4172549725828114,0.08847207125861667,7.574803142038387,8.236220331048578,8.645669067102506,1.07086592506412,0.16377928748169523,0.26388315860375433,0.6098587952263942,1.2523849775255826,0.9885018189218283,-0.15985880714732315
6,M,0.015689684315495306,2.354322624351259,3.3936429375786537,3.936924010402065,1.582601386050806,0.433222973905111,6.503937216974267,7.763779481755584,8.45669272738531,1.952755510411043,0.7362205182444157,0.31330824954984576,0.8075591590107598,2.4880122511778677,2.174704001628022,-0.3075591590107598
7,M,0.013798046341396492,2.307080791931832,2.992087362013524,3.3936429375786537,1.0865621456468215,0.16653919438296017,6.598425386832866,7.952755821472782,8.393700614146244,1.795275227313378,0.24724398779235468,0.26388315860375433,0.6592838861724857,2.0926115236091363,1.8287283650053818,-0.20928389809341463
9,I,1.0357090568324803,2.3307017081415458,4.267616837338054,4.574688748064331,2.243987039922785,-0.09933051379069102,7.07086623612586,8.614173010482974,8.960629633297836,1.8897633971719765,0.2403916562650732,0.21445806765766293,0.6098587952263942,1.4500853413099482,1.2356272736522853,-0.3598587952263942
13,I,0.703483589038828,2.3307017081415458,4.078649507660346,4.409342334596335,2.0786406264547894,-0.014954553741889143,6.724409613310997,8.425196670765775,8.771653293580638,2.0472436802696405,0.314371085887057,0.26388315860375433,0.6592838861724857,2.1914617055013186,1.9275785468975641,-0.5092838802120212


In [64]:
m31_Prop2filt

ID,BEST_METHOD,Chi_2(2),Mass_PDF_P25(2),Mass_PDF_Median(2),Mass_PDF_P75(2),MassIQR(2),SlugMass-m31BestMass(2),Age_PDF_P25(2),Age_PDF_Median(2),Age_PDF_P75(2),AgeIQR(2),SlugAge-m31BestAge(2),AV_PDF_P25(2),A_V_PDF_Median(2),A_V_PDF_P75(2),AvIQR(2),SlugAV-CMD_AV_Best(2)
int32,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,I,1.3330784569926594e-05,2.1889762108832644,2.732257283706675,4.149512256289487,1.9605360454062226,1.3984666695403951,6.094488480920338,6.440945103735201,8.551180897243908,2.4566924163235697,2.5300126523072795,0.46158352238812,1.1535347956333997,1.9443362507708621,1.4827527283827422,-1.1535347956333997
2,M,3.542105481810372e-05,2.1417343784638376,2.5905317864483943,3.582610267256362,1.4408758887925242,1.3883443274432072,6.125984537539872,6.661417500071932,8.45669272738531,2.3307081898454376,1.7385821184583419,0.41215843144202857,1.054684613741217,1.8949111598247705,1.482752728382742,0.09531536241692518
3,I,5.730100847666283e-06,2.0944925460444104,2.448806289190113,3.110191943062092,1.0156993970176815,0.6413986649572259,6.188976650778938,6.850393839789129,8.204724274429045,2.0157476236501077,1.718182019280695,0.3627333404959372,0.8569842499568513,1.5983606141482223,1.235627273652285,-0.8569842499568513
4,I,0.00017348730626736097,2.165355294673551,2.637773618867821,3.8660612617729244,1.7007059670993736,1.4879111192913586,6.094488480920338,6.409449047115668,8.173228217809513,2.0787397368891742,2.565230899783746,0.46158352238812,1.104109704687308,1.8949111598247705,1.4333276374366506,-1.104109704687308
5,M,1.811175454186711e-05,2.118113462254124,2.519669037819254,3.322780188949513,1.2046667266953892,0.891583222388876,6.157480594159405,6.661417500071932,8.173228217809513,2.0157476236501077,1.7385821184583419,0.41215843144202857,1.0052595227951253,1.6972107960404053,1.2850523645983767,-0.5552595347160543
6,M,1.4551837587492784e-05,2.165355294673551,2.6613945350775348,3.9133030941923512,1.7479477995188004,1.1654713764062299,6.094488480920338,6.472441160354734,8.48818878400484,2.3937003030845023,2.027558839645266,0.41215843144202857,1.104109704687308,2.043186432663045,1.6310280012210163,-0.6041097046873081
7,M,2.3518865145535927e-06,2.118113462254124,2.4724272053998266,3.228296524110659,1.1101830618565351,0.6861993509966577,6.157480594159405,6.75590566993053,8.173228217809513,2.0157476236501077,1.4440941393346067,0.3627333404959372,0.9064093409029427,1.6477857050943139,1.2850523645983767,-0.45640935282387163
9,I,1.0488987860859004e-05,2.165355294673551,2.614152702658108,3.6770939320952163,1.5117386374216655,1.5541336208892553,6.094488480920338,6.377952990496135,7.7322834251360515,1.6377949442157131,2.4766116762519115,0.46158352238812,1.0052595227951253,1.6972107960404053,1.2356272736522853,-0.7552595227951253
11,I,9.963258102986174e-05,2.165355294673551,2.7794991161261025,3.7479566807243567,1.582601386050806,0.6339043949212608,6.188976650778938,7.606299198657919,8.866141463439238,2.6771648126603003,1.232641421764444,0.3627333404959372,1.104109704687308,2.1420366145552276,1.7793032740592905,-1.104109704687308


In [65]:
m31_prop_all=hstack([m31_Prop6filt,m31_Prop5filt,
                     m31_Prop4filt,m31_Prop2filt])
m31_prop_all

ID_1,BEST_METHOD_1,Chi_2(6),Mass_PDF_P25(6),Mass_PDF_Median(6),Mass_PDF_P75(6),MassIQR(6),SlugMass-m31BestMass(6),Age_PDF_P25(6),Age_PDF_Median(6),Age_PDF_P75(6),AgeIQR(6),SlugAge-m31BestAge(6),AV_PDF_P25(6),A_V_PDF_Median(6),A_V_PDF_P75(6),AvIQR(6),SlugAV-CMD_AV_Best(6),ID_2,BEST_METHOD_2,Chi_2(5),Mass_PDF_P25(5),Mass_PDF_Median(5),Mass_PDF_P75(5),MassIQR(5),SlugMass-m31BestMass(5),Age_PDF_P25(5),Age_PDF_Median(5),Age_PDF_P75(5),AgeIQR(5),SlugAge-m31BestAge(5),AV_PDF_P25(5),A_V_PDF_Median(5),A_V_PDF_P75(5),AvIQR(5),SlugAV-CMD_AV_Best(5),ID_3,BEST_METHOD_3,Chi_2(4),Mass_PDF_P25(4),Mass_PDF_Median(4),Mass_PDF_P75(4),MassIQR(4),SlugMass-m31BestMass(4),Age_PDF_P25(4),Age_PDF_Median(4),Age_PDF_P75(4),AgeIQR(4),SlugAge-m31BestAge(4),AV_PDF_P25(4),A_V_PDF_Median(4),A_V_PDF_P75(4),AvIQR(4),SlugAV-CMD_AV_Best(4),ID_4,BEST_METHOD_4,Chi_2(2),Mass_PDF_P25(2),Mass_PDF_Median(2),Mass_PDF_P75(2),MassIQR(2),SlugMass-m31BestMass(2),Age_PDF_P25(2),Age_PDF_Median(2),Age_PDF_P75(2),AgeIQR(2),SlugAge-m31BestAge(2),AV_PDF_P25(2),A_V_PDF_Median(2),A_V_PDF_P75(2),AvIQR(2),SlugAV-CMD_AV_Best(2)
int32,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,I,2.7985711649757494,2.307080791931832,4.220375004918627,4.480205083225476,2.173124291293644,-0.08965105167155674,7.07086623612586,8.67716512372204,8.960629633297836,1.8897633971719765,0.29379263232044117,0.26388315860375433,0.6098587952263942,1.6477857050943139,1.3839025464905594,-0.6098587952263942,1,I,0.3194347026193026,2.354322624351259,4.0550285914506325,4.432963250806049,2.07864062645479,0.07569536179643777,7.07086623612586,8.393700614146244,8.929133576678304,1.8582673405524437,0.5772571418962364,0.26388315860375433,0.6592838861724857,1.7466358869864964,1.4827527283827422,-0.6592838861724857,1,I,2.5684339984198012,2.401564456770686,4.267616837338054,4.527446915644903,2.125882458874217,-0.136892884090984,7.1338583493649255,8.708661180341572,8.99212568991737,1.8582673405524437,0.26229657570090836,0.26388315860375433,0.6098587952263942,1.548935523202131,1.2850523645983767,-0.6098587952263942,1,I,1.3330784569926594e-05,2.1889762108832644,2.732257283706675,4.149512256289487,1.9605360454062226,1.3984666695403951,6.094488480920338,6.440945103735201,8.551180897243908,2.4566924163235697,2.5300126523072795,0.46158352238812,1.1535347956333997,1.9443362507708621,1.4827527283827422,-1.1535347956333997
4,I,2.1949285655453195,2.2834598757221185,4.196754088708913,4.432963250806049,2.1495033750839303,-0.07106935054973373,7.07086623612586,8.645669067102506,8.960629633297836,1.8897633971719765,0.3290108797969076,0.21445806765766293,0.6098587952263942,1.6477857050943139,1.433327637436651,-0.6098587952263942,4,I,0.22185885679230938,2.3779435405609726,3.9605449266117785,4.362100502176908,1.9841569616159354,0.1651398115474012,7.039370179506327,8.299212444287644,8.866141463439238,1.8267712839329109,0.6754675026117702,0.26388315860375433,0.6592838861724857,1.7466358869864964,1.4827527283827422,-0.6592838861724857,2,M,0.28868409604292605,2.259838959512405,3.440884769998081,3.818819429353497,1.5589804698410923,0.5379913438935207,7.543307085418854,8.267716387668111,8.740157236961105,1.1968501515422512,0.13228323086216243,0.26388315860375433,0.6592838861724857,1.4500853413099482,1.186202182706194,0.4907160899856564,2,M,3.542105481810372e-05,2.1417343784638376,2.5905317864483943,3.582610267256362,1.4408758887925242,1.3883443274432072,6.125984537539872,6.661417500071932,8.45669272738531,2.3307081898454376,1.7385821184583419,0.412

In [66]:
m31_prop_all.rename_column('ID_1', 'ID(6)')
m31_prop_all.rename_column('BEST_METHOD_1', 'BEST_METHOD(6)')
m31_prop_all.rename_column('ID_2', 'ID(5)')
m31_prop_all.rename_column('BEST_METHOD_2', 'BEST_METHOD(5)')
m31_prop_all.rename_column('ID_3', 'ID(4)')
m31_prop_all.rename_column('BEST_METHOD_3', 'BEST_METHOD(4)')
m31_prop_all.rename_column('ID_4', 'ID(2)')
m31_prop_all.rename_column('BEST_METHOD_4', 'BEST_METHOD(2)')
m31_prop_all

ID(6),BEST_METHOD(6),Chi_2(6),Mass_PDF_P25(6),Mass_PDF_Median(6),Mass_PDF_P75(6),MassIQR(6),SlugMass-m31BestMass(6),Age_PDF_P25(6),Age_PDF_Median(6),Age_PDF_P75(6),AgeIQR(6),SlugAge-m31BestAge(6),AV_PDF_P25(6),A_V_PDF_Median(6),A_V_PDF_P75(6),AvIQR(6),SlugAV-CMD_AV_Best(6),ID(5),BEST_METHOD(5),Chi_2(5),Mass_PDF_P25(5),Mass_PDF_Median(5),Mass_PDF_P75(5),MassIQR(5),SlugMass-m31BestMass(5),Age_PDF_P25(5),Age_PDF_Median(5),Age_PDF_P75(5),AgeIQR(5),SlugAge-m31BestAge(5),AV_PDF_P25(5),A_V_PDF_Median(5),A_V_PDF_P75(5),AvIQR(5),SlugAV-CMD_AV_Best(5),ID(4),BEST_METHOD(4),Chi_2(4),Mass_PDF_P25(4),Mass_PDF_Median(4),Mass_PDF_P75(4),MassIQR(4),SlugMass-m31BestMass(4),Age_PDF_P25(4),Age_PDF_Median(4),Age_PDF_P75(4),AgeIQR(4),SlugAge-m31BestAge(4),AV_PDF_P25(4),A_V_PDF_Median(4),A_V_PDF_P75(4),AvIQR(4),SlugAV-CMD_AV_Best(4),ID(2),BEST_METHOD(2),Chi_2(2),Mass_PDF_P25(2),Mass_PDF_Median(2),Mass_PDF_P75(2),MassIQR(2),SlugMass-m31BestMass(2),Age_PDF_P25(2),Age_PDF_Median(2),Age_PDF_P75(2),AgeIQR(2),SlugAge-m31BestAge(2),AV_PDF_P25(2),A_V_PDF_Median(2),A_V_PDF_P75(2),AvIQR(2),SlugAV-CMD_AV_Best(2)
int32,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,bytes1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,I,2.7985711649757494,2.307080791931832,4.220375004918627,4.480205083225476,2.173124291293644,-0.08965105167155674,7.07086623612586,8.67716512372204,8.960629633297836,1.8897633971719765,0.29379263232044117,0.26388315860375433,0.6098587952263942,1.6477857050943139,1.3839025464905594,-0.6098587952263942,1,I,0.3194347026193026,2.354322624351259,4.0550285914506325,4.432963250806049,2.07864062645479,0.07569536179643777,7.07086623612586,8.393700614146244,8.929133576678304,1.8582673405524437,0.5772571418962364,0.26388315860375433,0.6592838861724857,1.7466358869864964,1.4827527283827422,-0.6592838861724857,1,I,2.5684339984198012,2.401564456770686,4.267616837338054,4.527446915644903,2.125882458874217,-0.136892884090984,7.1338583493649255,8.708661180341572,8.99212568991737,1.8582673405524437,0.26229657570090836,0.26388315860375433,0.6098587952263942,1.548935523202131,1.2850523645983767,-0.6098587952263942,1,I,1.3330784569926594e-05,2.1889762108832644,2.732257283706675,4.149512256289487,1.9605360454062226,1.3984666695403951,6.094488480920338,6.440945103735201,8.551180897243908,2.4566924163235697,2.5300126523072795,0.46158352238812,1.1535347956333997,1.9443362507708621,1.4827527283827422,-1.1535347956333997
4,I,2.1949285655453195,2.2834598757221185,4.196754088708913,4.432963250806049,2.1495033750839303,-0.07106935054973373,7.07086623612586,8.645669067102506,8.960629633297836,1.8897633971719765,0.3290108797969076,0.21445806765766293,0.6098587952263942,1.6477857050943139,1.433327637436651,-0.6098587952263942,4,I,0.22185885679230938,2.3779435405609726,3.9605449266117785,4.362100502176908,1.9841569616159354,0.1651398115474012,7.039370179506327,8.299212444287644,8.866141463439238,1.8267712839329109,0.6754675026117702,0.26388315860375433,0.6592838861724857,1.7466358869864964,1.4827527283827422,-0.6592838861724857,2,M,0.28868409604292605,2.259838959512405,3.440884769998081,3.818819429353497,1.5589804698410923,0.5379913438935207,7.543307085418854,8.267716387668111,8.740157236961105,1.1968501515422512,0.13228323086216243,0.26388315860375433,0.6592838861724857,1.4500853413099482,1.186202182706194,0.4907160899856564,2,M,3.542105481810372e-05,2.1417343784638376,2.5905317864483943,3.582610267256362,1.4408758887925242,1.3883443274432072,6.125984537539872,6.661417500071932,8.45669272738531,2.3307081898454376,1.73858211845834

In [67]:
m31_prop_all.write('/Volumes/My_Passport_for_Mac/M33_M31_LGCS_Files/M31-Andromeda/Prop_Tables/M31_Prop_All.fits')
